In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer #bag of words
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


# 1. Carregar base de dados

Primeiramente, vamos carregar uma base de dados do Kaggle com mais 700 mil tweets em português. Os tweets estão "rotulados" como positivos e negativos:


In [4]:
data = pd.read_csv('NoThemeTweets.csv')
data = data[['tweet_text','sentiment']]
data.head()

,tweet_text,sentiment
0,@Tixaa23 14 para eu ir :),Positivo
1,@drexalvarez O meu like eu já dei na época :),Positivo
2,Eu só queria conseguir comer alguma coisa pra ...,Positivo
3,:D que lindo dia !,Positivo
4,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Positivo


# 2. Limpeza da Base


Com a base de dados carregada, precisamos limpar os caracteres especiais dos tweets:

In [14]:
data['tweet_text'] = data['tweet_text'].apply(lambda x: x.lower())
data['tweet_text'] = data['tweet_text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positivo'])
print(data[ data['sentiment'] == 'Negativo'])

                                               tweet_text sentiment
0                                  tixaa23 14 para eu ir   Positivo
1                drexalvarez o meu like eu j dei na poca   Positivo
2       eu s queria conseguir comer alguma coisa pra p...  Positivo
3                                        d que lindo dia   Positivo
4       primo_resmungao pq da pr jeito uma ofe a ha q ...  Positivo
...                                                   ...       ...
785809                           acordar 8 horas  to bom   Positivo
785810  mayckcunha ol mayck voc j  cliente claro caso ...  Positivo
785811  opa tava na merda mm e fiquei logo mais feliz ...  Positivo
785812            andrebraga2806 foi como a tua lealdade   Positivo
785813  feliz dia das crianas de hoje e de ontem  http...  Positivo

[263107 rows x 2 columns]
                                               tweet_text sentiment
46786                    lugmvdua e no estou a trabalhar   Negativo
47753              vo

# 3. Tokenizer

Em seguida, definimos o número máximo de recursos como 2.000 e usamos o Tokenizer para vetorizar e converter os tweets em sequências de textos, para conseguirmos utilizá-los como entrada na rede:

In [6]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['tweet_text'].values)
X = tokenizer.texts_to_sequences(data['tweet_text'].values)
X = pad_sequences(X)

# 4. Rede LSTM

O próximo passo é montar uma rede LSTM. Definimos as variáveis **embed_dim*,* lstm_out*,* batch_size*,* droupout_x** como hiperparâmetros e usamos o *softmax* como função de ativação, uma vez que a nossa rede está usando a entropia cruzada categórica e o *softmax* é o método de ativação certo para isso:

In [7]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 84, 128)           256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 84, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


Em seguida, declaramos as bases de treino e teste para rodar o modelo:

In [8]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(526495, 84) (526495, 2)
(259319, 84) (259319, 2)


In [10]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)

16453/16453 - 4709s - loss: 0.4200 - accuracy: 0.8019


# 5. Mensuração do Modelo

Para mensurar o modelo, extraímos uma base de validação e calculamos o score e a acurácia:

In [11]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

8057/8057 - 159s - loss: 0.4020 - accuracy: 0.8122
score: 0.40
acc: 0.81


Por último, medimos o número de acertos da rede. Uma vez que a base de dados contém muito mais tweets negativos do que positivos (67% vs 33%), a rede teve uma acurácia muito maior ao tentar prever os tweets negativos do que os positivos (91,75% vs 59,80%):
 

In [12]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 1s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1